# TITANIC

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn import linear_model
from sklearn.cluster import KMeans
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pred_aceptable = np.vectorize(lambda x: int(round(x-x)) if x<0.5 else int(round(x+(1-x))))

ModuleNotFoundError: No module named 'matplotlib.pyplot'

In [ ]:
df = pd.read_csv('train.csv')
test_original = pd.read_csv('test.csv')

In [ ]:
df.head()

In [ ]:
#Division en TRAIN = X_train y VALIDATION = X_test
X = df.drop(['Survived','PassengerId'], axis=1)

y = df['Survived']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=382)

print(X_train.shape[0], X_test.shape[0])

# Analisis de variables

In [ ]:
X_train.isnull().sum()

In [ ]:
#Cabin - se elimina porque tiene mas nulos que variables completas
X_train.drop(['Cabin'], axis=1,inplace=True)

X_test.drop(['Cabin'], axis=1,inplace=True)

test_original.drop(['Cabin'], axis=1,inplace=True)

Age is an important factor in the survival, what we can deduce from these is that:
- Age is between 0 and 10 from any class you are likely to survived
- > 10 the number of children increses for clases 2 and 3, and they survived

In [ ]:
#Age - Sex vs Survived
f,ax=plt.subplots(1,2,figsize=(18,8))
sb.violinplot('Pclass','Age',hue='Survived',data=df,split=True,ax=ax[0])
ax[0].set_title('PClass and Age vs Survived')
ax[0].set_yticks(range(0,110,5))
sb.violinplot("Sex","Age", hue="Survived", data=df,split=True,ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0,110,5))
plt.show()

In [ ]:
#Text analisis of variable name to impute mean age
X_train['Initial']=0
for i in X_train:
    X_train['Initial']=X_train.Name.str.extract('([A-Za-z]+)\.') #extracting Name initials

pd.crosstab(X_train.Initial,X_train.Sex).T.style.background_gradient(cmap='summer_r')

X_train['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess',\
                        'Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss',\
                        'Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)

X_train.groupby('Initial')['Age'].mean()

X_train.loc[(X_train.Age.isnull()) & (X_train.Initial=='Mr'),'Age']= 33
X_train.loc[(X_train.Age.isnull()) & (X_train.Initial=='Mrs'),'Age']=36
X_train.loc[(X_train.Age.isnull()) & (X_train.Initial=='Master'),'Age']=5
X_train.loc[(X_train.Age.isnull()) & (X_train.Initial=='Miss'),'Age']=22
X_train.loc[(X_train.Age.isnull()) & (X_train.Initial=='Other'),'Age']=46

X_train.Age.isnull().any()

#Se elimina la variable Initial, ya que se genero solo para el calculo
X_train.drop(['Initial'], axis=1,inplace=True)



In [ ]:
#Se repite lo mismo para TEST y Validation
X_test['Initial']=0
for i in X_test:
    X_test['Initial']=X_test.Name.str.extract('([A-Za-z]+)\.') #extracting Name initials



X_test['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess',\
                        'Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss',\
                        'Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)


X_test.loc[(X_test.Age.isnull()) & (X_test.Initial=='Mr'),'Age']= 33
X_test.loc[(X_test.Age.isnull()) & (X_test.Initial=='Mrs'),'Age']=36
X_test.loc[(X_test.Age.isnull()) & (X_test.Initial=='Master'),'Age']=5
X_test.loc[(X_test.Age.isnull()) & (X_test.Initial=='Miss'),'Age']=22
X_test.loc[(X_test.Age.isnull()) & (X_test.Initial=='Other'),'Age']=46

X_test.drop(['Initial'], axis=1,inplace=True)

test_original['Initial']=0
for i in test_original:
    test_original['Initial']=test_original.Name.str.extract('([A-Za-z]+)\.') #extracting Name initials



test_original['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess',\
                        'Jonkheer','Col','Rev','Capt','Sir','Don'],['Miss',\
                        'Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr'],inplace=True)



test_original.loc[(test_original.Age.isnull()) & (test_original.Initial=='Mr'),'Age']= 33
test_original.loc[(test_original.Age.isnull()) & (test_original.Initial=='Mrs'),'Age']=36
test_original.loc[(test_original.Age.isnull()) & (test_original.Initial=='Master'),'Age']=5
test_original.loc[(test_original.Age.isnull()) & (test_original.Initial=='Miss'),'Age']=22
test_original.loc[(test_original.Age.isnull()) & (test_original.Initial=='Other'),'Age']=46

test_original.drop(['Initial'], axis=1,inplace=True)

In [ ]:
X_train.describe()

In [ ]:
#Se elimina la varibale NAME porque no aporta
X_train.drop(['Name'], axis=1,inplace=True)

X_test.drop(['Name'], axis=1,inplace=True)

test_original.drop(['Name'], axis=1,inplace=True)

In [ ]:
#Se convierte la variable SEX a numerico, dado que para los modelos como texto no se puede utilizar, 
#y es una variable que aporta mucha informacion
X_train['Sex'].replace(['male','female',''], [1,2,3],inplace=True)

X_test['Sex'].replace(['male','female',''], [1,2,3],inplace=True)

test_original['Sex'].replace(['male','female',''], [1,2,3],inplace=True)

In [ ]:
#Se elimina la varibale Ticket porque no aporta
X_train.drop(['Ticket'], axis=1,inplace=True)

X_test.drop(['Ticket'], axis=1,inplace=True)

test_original.drop(['Ticket'], axis=1,inplace=True)

In [ ]:
#Count por Embarked - Se genera un id para cada puerto y se reemplaza por las letras
# 1 = C
# 2 = S
# 3 = Q
# 4 = NA

X_train['Embarked'].replace(['C','S','Q',''], [1,2,3,4],inplace=True)

X_test['Embarked'].replace(['C','S','Q',''], [1,2,3,4],inplace=True)

test_original['Embarked'].replace(['C','S','Q',''], [1,2,3,4],inplace=True)


X_train.loc[(X_train.Embarked.isnull()) ,'Embarked']= 4
X_test.loc[(X_test.Embarked.isnull()) ,'Embarked']= 4
test_original.loc[(test_original.Embarked.isnull()) ,'Embarked']= 4


In [ ]:
print(X_train.shape, X_test.shape,test_original.shape)

In [ ]:
X_train.head()

In [ ]:
test_original.head()

In [ ]:
X_test.head()

In [ ]:
df.describe()

# MODELOS

In [ ]:
#Se crea un dataframe para guardar los resultados de los distintos modelos
results = pd.DataFrame()
col_names =  ['Modelo','Acuracy']
results  = pd.DataFrame(columns = col_names)
#res = accuracy_score(y_test, y_pred)
#results = results.append([{'Modelo':'Arbol - cluster lat-long-sup outliers-barrio_match - max depth = 6', 'Acuracy':res }], ignore_index=True)
results.head(10)

In [ ]:
X_train.describe()

ARBOL DE DECISION - ANALISIS DE MEJOR DEPTH
MAX_DEPTH = 2

In [ ]:
#se inicializan dos listas para poder generar sus metodos
rmses_train = []
rmses_test = []

for max_depth in range(1,10, 1):
    regressor = DecisionTreeRegressor(random_state=0, max_depth=max_depth)
    regressor.fit(X_train, y_train)
    
    y_pred_train = regressor.predict(X_train)
    y_pred_train = pred_aceptable(y_pred_train)
    
    y_pred_test = regressor.predict(X_test)
    y_pred_test = pred_aceptable(y_pred_test)
    
    rmse_train = accuracy_score(y_train, y_pred_train)
    rmse_test = accuracy_score(y_test, y_pred_test)
    
    rmses_train.append(rmse_train)
    rmses_test.append(rmse_test)

In [ ]:
%matplotlib inline 
plt.plot(range(1,10, 1), rmses_train, label='Accuracy Training')
plt.plot(range(1,10, 1), rmses_test, label='Accuracy Testing')
plt.ylim((0, 1))
plt.legend(loc="best")
plt.title("Accuracy Training vs Accuracy Testing para árboles de decisión")
plt.show()

In [ ]:
regressor_Arb = DecisionTreeRegressor(random_state=0, max_depth=6)
regressor_Arb.fit(X_train, y_train)
y_pred = regressor_Arb.predict(X_test)
y_pred = pred_aceptable(y_pred)
rmse = accuracy_score(y_test, y_pred)
print(np.round(rmse, 2))
results = results.append([{'Modelo':'Arbol - max depth = 2', 'Acuracy':rmse }], ignore_index=True)
results.head(10)

KNN -- Se analiza cual es el valor de vecinos optimo


In [ ]:
rmses_train = []
rmses_test = []
for nn in range(1,30, 1):
    regressor = KNeighborsRegressor(n_neighbors=nn)
    regressor.fit(X_train, y_train)
    
    y_pred_train = regressor.predict(X_train)
    y_pred_train = pred_aceptable(y_pred_train)
    
    y_pred_test = regressor.predict(X_test)
    y_pred_test = pred_aceptable(y_pred_test)
    
    rmse_train = accuracy_score(y_train, y_pred_train)
    rmse_test = accuracy_score(y_test, y_pred_test)
    
    rmses_train.append(rmse_train)
    rmses_test.append(rmse_test)

In [ ]:
plt.plot(range(1,30, 1), rmses_train, label='Acurracy Training')
plt.plot(range(1,30, 1), rmses_test, label='Acurracy Testing')
plt.ylim((0, 1))
plt.legend(loc="best")
plt.title("Acurracy Training vs Acurracy Testing para KNN")
plt.show()

In [ ]:
##KNN
regressor = KNeighborsRegressor(n_neighbors=8)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
y_pred = pred_aceptable(y_pred)
rmse = accuracy_score(y_test, y_pred)
print(np.round(rmse, 2))
results = results.append([{'Modelo':'KNN - 8', 'Acuracy':rmse }], ignore_index=True)
results.head(10)

LINEAL

In [ ]:
#LINEAL
regressor = linear_model.LinearRegression()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
y_pred = pred_aceptable(y_pred)
rmse = accuracy_score(y_test, y_pred)
print(np.round(rmse, 2))
results = results.append([{'Modelo':'Lineal', 'Acuracy':rmse }], ignore_index=True)
results.head(10)

RIDGE

In [ ]:
regressor = linear_model.Ridge(alpha=0.5)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
y_pred = pred_aceptable(y_pred)
rmse = accuracy_score(y_test, y_pred)
print(np.round(rmse, 2))
results = results.append([{'Modelo':'Ridge', 'Acuracy':rmse }], ignore_index=True)
results.head(10)

LASSO

In [ ]:
regressor = linear_model.LassoLars(alpha=.1)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
y_pred = pred_aceptable(y_pred)
rmse = accuracy_score(y_test, y_pred)
print(np.round(rmse, 2))
results = results.append([{'Modelo':'Lasso ', 'Acuracy':rmse }], ignore_index=True)
results.head(20)

BayesianRidge

In [ ]:
regressor = linear_model.BayesianRidge()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
y_pred = pred_aceptable(y_pred)
rmse = accuracy_score(y_test, y_pred)
print(np.round(rmse, 2))
results = results.append([{'Modelo':'BayesianRidge', 'Acuracy':rmse }], ignore_index=True)
results.head(10)

In [ ]:
results.sort_values(by='Acuracy', axis=0, ascending=False, inplace=False).head()

# PREDICCION PARA TEST ORIGINAL con el mejor modelo ARBOL CON PROFUNDIDAD DE 2

In [ ]:
Xtest_original = test_original.drop(['PassengerId'], axis=1)

ID_test_original = test_original['PassengerId']

In [ ]:
#regressor_Arb = DecisionTreeRegressor(random_state=0, max_depth=6)
#regressor_Arb.fit(X_train, y_train)
y_pred_final = regressor_Arb.predict(Xtest_original)
y_pred_final = pred_aceptable(y_pred_final)


y_pred_final =pd.DataFrame(y_pred_final,columns=["Survived"])

In [ ]:
df_out = pd.concat([ID_test_original,y_pred_final], axis=1 )
df_out.to_csv("output/arbol_python.csv",sep=",",index=False)

# SEGUNDO INTENTO

In [ ]:
X_train.head()

Analisis de outliers para Fare

In [ ]:
plt.boxplot(X_train.Fare)

In [ ]:
#Se eliminan los casos mayores a 300
X_train = X_train[ (X_train.Fare < X_train.Fare.quantile(0.9))]

In [ ]:
X_train.Fare.describe()

In [ ]:
X_train.drop(['cluster'], axis=1,inplace=True)

In [ ]:
X_train.describe()

In [ ]:
#Se eliminan las filas en y_train que se eliminaron en X_train
y_train = y_train.loc[X_train.index]

Modelos

In [ ]:
#se inicializan dos listas para poder generar sus metodos
rmses_train = []
rmses_test = []

for max_depth in range(1,30, 1):
    regressor = DecisionTreeRegressor(random_state=0, max_depth=max_depth)
    regressor.fit(X_train, y_train)
    
    y_pred_train = regressor.predict(X_train)
    y_pred_train = pred_aceptable(y_pred_train)
    
    y_pred_test = regressor.predict(X_test)
    y_pred_test = pred_aceptable(y_pred_test)
    
    rmse_train = accuracy_score(y_train, y_pred_train)
    rmse_test = accuracy_score(y_test, y_pred_test)
    
    rmses_train.append(rmse_train)
    rmses_test.append(rmse_test)
    
plt.plot(range(1,30, 1), rmses_train, label='Acurracy Training')
plt.plot(range(1,30, 1), rmses_test, label='Acurracy Testing')
plt.ylim((0, 1))
plt.legend(loc="best")
plt.title("Acurracy Training vs Acurracy Testing para Arbol")
plt.show()

In [ ]:
regressor_Arb = DecisionTreeRegressor(random_state=0, max_depth=8)
regressor_Arb.fit(X_train, y_train)
y_pred = regressor_Arb.predict(X_test)
y_pred = pred_aceptable(y_pred)
rmse = accuracy_score(y_test, y_pred)
print(np.round(rmse, 2))
results = results.append([{'Modelo':'Arbol - segundo intento -max depth = 8', 'Acuracy':rmse }], ignore_index=True)
results.head(10)

In [ ]:
rmses_train = []
rmses_test = []
for nn in range(1,30, 1):
    regressor = KNeighborsRegressor(n_neighbors=nn)
    regressor.fit(X_train, y_train)
    
    y_pred_train = regressor.predict(X_train)
    y_pred_train = pred_aceptable(y_pred_train)
    
    y_pred_test = regressor.predict(X_test)
    y_pred_test = pred_aceptable(y_pred_test)
    
    rmse_train = accuracy_score(y_train, y_pred_train)
    rmse_test = accuracy_score(y_test, y_pred_test)
    
    rmses_train.append(rmse_train)
    rmses_test.append(rmse_test)
    
plt.plot(range(1,30, 1), rmses_train, label='Acurracy Training')
plt.plot(range(1,30, 1), rmses_test, label='Acurracy Testing')
plt.ylim((0, 1))
plt.legend(loc="best")
plt.title("Acurracy Training vs Acurracy Testing para KNN")
plt.show()

In [ ]:
##KNN
regressor = KNeighborsRegressor(n_neighbors=8)
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)
y_pred = pred_aceptable(y_pred)
rmse = accuracy_score(y_test, y_pred)
print(np.round(rmse, 2))
results = results.append([{'Modelo':'KNN - 8-seg intento', 'Acuracy':rmse }], ignore_index=True)
results.head(10)